In [1]:
%run '000_shared.ipynb'

In [157]:
class FuncsUtil:
    @staticmethod
    def agg_identity(inputz, weightz=None):
        return inputz
    @staticmethod
    def agg_sum(inputz, weightz):
        #print("sum: ", np.dot(inputz, weightz) )
        return np.dot(inputz, weightz)
    
    @staticmethod
    def act_relu(inputz):
        #print("relu: ", max(0, inputz))
        return max(0, inputz)
    @staticmethod
    def act_sigmoid(inputz):
        pass
    
    @staticmethod
    def out_softmax( inputz):
        return (inputz)
    def out_softmax_todo(node_index, inputz):
        return np.exp( inputz[node_index] ) / np.sum( np.exp(inputz) )
    
    @staticmethod
    def out_logit(x):
        return 1.0/(1 + np.exp(-x))
    
    @staticmethod 
    def deriv_logit(x):
        return FuncsUtil.out_logit(x) * (1 - FuncsUtil.out_logit(x))
    
    @staticmethod
    def loss_hinge(predicted, ylabelz):
        l = (predicted - ylabelz)
        print("losses: ",l)
        return l
    
        
class Node:
    def __init__(self, value=None):
        self.value = value
        
    def __str__(self):
        return f"[ {self.value} ]"
    
class HiddenNode(Node): 
    def __init__(self, inputs_agg_func, activation_func):
        Node.__init__(self)
        self.inputs_agg_func = inputs_agg_func
        self.activation_func = activation_func
        
    def pass_inputs(self, inputs_array, weights_array):
        self.value = self.activation_func( 
                        self.inputs_agg_func(
                            inputs_array, weights_array) )

class Layer:
    TYPE_INPUT = 1
    TYPE_HIDDEN = 2
    TYPE_OUTPUT = 3 
    
    def __init__(self, layer_type, n_nodes, agg_func=None, activ_func=None ): #layer types = 1.input, 2.hidden, 3.output
        self.layer_type = layer_type 
        self.nodes = []
        self.init_nodes(n_nodes, agg_func, activ_func)
        
    #layer types = 1.input, 2.hidden, 3.output
    def init_nodes(self, n_nodes, agg_func, activ_func):
        def init_hidden_or_output_nodes():
            for i in range(n_nodes): 
                self.nodes.append( HiddenNode(agg_func, activ_func)) 
        def init_input_nodes():
            for i in range(n_nodes):
                self.nodes.append(Node())
                
        if self.layer_type == Layer.TYPE_INPUT:
            init_input_nodes()
            self.weights = np.ones(n_nodes)
        else:
            init_hidden_or_output_nodes() 
            self.weightz = np.random.rand(n_nodes)
            
    @property 
    def outputs(self):
        return np.array([ n.value for n in self.nodes ]) 
            
    
    def pass_inputs(self, inputs_array):
        if self.layer_type == Layer.TYPE_INPUT:
            for i in range( len(self.nodes)):
                self.nodes[i].value = inputs_array[i]
        else:
            for node in self.nodes:
                node.pass_inputs( inputs_array, self.weightz)
    
            
    def back_propagate(self):
        pass
    
    def __str__(self): 
        n = []
        for ni in self.nodes:
            n.append( str(ni))
        n = ", ".join( n )
        return f"<< {len(self.nodes)}: {n} >>"
    
    
class Network: ##TODO: Is Computational Graph
    def __init__(self, n_inputs, n_hidden_layers, n_outputs, y_labelz, 
                 n_hidden_nodes=None,
                 hidden_width_decay_rate = 0.8, 
                 agg_func=FuncsUtil.agg_sum, activ_func=FuncsUtil.act_relu, ouput_func=FuncsUtil.act_relu):
        
        self.n_inputs = n_inputs
        self.n_hidden_layers = n_hidden_layers 
        self.y_labelz = y_labelz
        
        self.input_layer = Layer(Layer.TYPE_INPUT, n_inputs )
        
        #<< TODO: fix output layer  
        self.output_layer = Layer(Layer.TYPE_OUTPUT, n_outputs, agg_func, ouput_func )
        self.output_losses = np.array([])
        
        self.hidden_layers = []
        self.accuracy = 0
        nh_nodez = (n_inputs*1) if n_hidden_nodes is None else n_hidden_nodes
        for i in range(n_hidden_layers):
            nh = nh_nodez #TODO: if (i == 0) or (i == (n_hidden_layers - 1)) else int(nh_nodez*hidden_width_decay_rate)
            self.hidden_layers.append( Layer(Layer.TYPE_HIDDEN, nh, agg_func, activ_func) )
            
        
        
    def set_inputs(self, inputz_array):
        self.input_layer.pass_inputs(inputz_array)
        
    ## TODO: Implement MLP 
    def train(self, epochs=10, learning_rate=0.1):
        for t in range(epochs):  
            ## 1. Hidden Layers 
            x = self.input_layer.outputs  
            for l in self.hidden_layers:                           
                ## Forward Pass
                l.pass_inputs(x)
                x = l.outputs
                print(f'{t} epoch train_out: ',x)
                ## Backpropagate update weights << TODO
                print(f'l.weightz = {l.weightz}')
                losses = FuncsUtil.loss_hinge(x, self.y_labelz)
                l.weightz = l.weightz + (learning_rate * losses )
            
            ## 2. Output Layer 
            self.output_layer.pass_inputs(x)            
            ## Backpropagate << TODO +fix output layer <<< when to update weights             
            self.output_losses = FuncsUtil.loss_hinge(self.output_layer.outputs, self.y_labelz)
            self.output_layer.weightz = self.output_layer.weightz + (learning_rate * self.output_losses )
        
            ## 3. Convergence & Accuracy << TODO: fix with step 2 above and output layer 
            print(f"Predicted: {self.output_layer.outputs}")
            self.accuracy = np.mean( self.output_layer.outputs - self.y_labelz ) 
        
    def __str__(self):
        h = f"Inputs: {self.n_inputs}\nHidden Layers: {self.n_hidden_layers}\n{'-'*13}"
        l = [f"I:\t{str(self.input_layer)}"] 
        for hl in self.hidden_layers:
            l.append( f"H:\t{str(hl)}"  )
        l.append( f"O:\t{str(self.output_layer)}"  )
        l2 = '\n'.join(l)
        return f"{h}\n{l2}\nOutput Weights:{self.output_layer.weightz}\nMean output loss: {np.mean(self.output_losses)}\nMean accuracy: {self.accuracy}"

In [160]:
x = [10, -30, -40, 20]
y = [1, 0, 0, 1]
n_out = len(x)
n_hidden_l = 1

net = Network( len(x), n_hidden_l, n_out, y)
net.set_inputs(x)
print(net, "\n\n")

net.train(3, 0.03)
print("\n\n",net)


Inputs: 4
Hidden Layers: 1
-------------
I:	<< 4: [ 10 ], [ -30 ], [ -40 ], [ 20 ] >>
H:	<< 4: [ None ], [ None ], [ None ], [ None ] >>
O:	<< 4: [ None ], [ None ], [ None ], [ None ] >>
Output Weights:[0.31985145 0.30301844 0.19034373 0.3788168 ]
Mean output loss: nan
Mean accuracy: 0 


0 epoch train_out:  [0 0 0 0]
l.weightz = [0.29546406 0.69581448 0.6222889  0.21716072]
losses:  [-1  0  0 -1]
losses:  [-1  0  0 -1]
Predicted: [0 0 0 0]
1 epoch train_out:  [0 0 0 0]
l.weightz = [0.26546406 0.69581448 0.6222889  0.18716072]
losses:  [-1  0  0 -1]
losses:  [-1  0  0 -1]
Predicted: [0 0 0 0]
2 epoch train_out:  [0 0 0 0]
l.weightz = [0.23546406 0.69581448 0.6222889  0.15716072]
losses:  [-1  0  0 -1]
losses:  [-1  0  0 -1]
Predicted: [0 0 0 0]


 Inputs: 4
Hidden Layers: 1
-------------
I:	<< 4: [ 10 ], [ -30 ], [ -40 ], [ 20 ] >>
H:	<< 4: [ 0 ], [ 0 ], [ 0 ], [ 0 ] >>
O:	<< 4: [ 0 ], [ 0 ], [ 0 ], [ 0 ] >>
Output Weights:[0.22985145 0.30301844 0.19034373 0.2888168 ]
Mean output loss

In [119]:
np.random.rand(3)

array([0.33611357, 0.93407836, 0.47403815])